In [1]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import os 

# Paths

google drive: https://drive.google.com/drive/folders/1NcerEtJUn6eULDLdu2l-WPdzvTTw6mFE?usp=sharing

In [2]:
data_path = os.path.dirname(os.getcwd()) + '/data/'

train_csv = pd.read_csv(data_path + '/aggregation/AggBERT_train.csv')
test_csv = pd.read_csv(data_path + '/aggregation/AggBERT_test.csv')

In [3]:
agg = np.eye(2)
agg_data_train = np.array(train_csv['label'])
agg_data_test = np.array(test_csv['label'])

AA = ['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
nb_values = np.eye(20)
translate = lambda x: AA.index(x)
raw_seq_train = np.array(train_csv['Sequence'])
raw_seq_test = np.array(test_csv['Sequence'])
sequences_train = np.array([list(map(translate, seq)) for seq in raw_seq_train])
sequences_test = np.array([list(map(translate, seq)) for seq in raw_seq_test])
one_hot_seq_train = np.array([nb_values[seq] for seq in sequences_train])
one_hot_seq_test = np.array([nb_values[seq] for seq in sequences_test])

data_train = np.empty((len(sequences_train), 6, 97))
data_train[:, :, 50:53] = 1
data_test = np.empty((len(sequences_test), 6, 97))
data_test[:, :, 50:53] = 1
names_train = []
names_test = []
for i in range(len(one_hot_seq_train)):
    data_train[i, 0, 95:97] = agg[agg_data_train[i]]
    data_train[i, :, :20] = one_hot_seq_train[i]
    names_train.append(f'AGGTrain{i:04}')
for i in range(len(one_hot_seq_test)):
    data_test[i, 0, 95:97] = agg[agg_data_test[i]]
    data_test[i, :, :20] = one_hot_seq_test[i]
    names_test.append(f'AGGTest{i:04}')

Traceback (most recent call last):
  File "/Users/Eck00018/Documents/PhD/dea_chapter_multi_task/chapter-multi-task/.conda/lib/python3.11/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_vars.py", line 624, in change_attr_expression
    value = eval(expression, frame.f_globals, frame.f_locals)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<string>", line 1, in <module>
NameError: name 'array' is not defined


In [4]:
np.savez_compressed(data_path + '/aggregation/Train_aggregation.npz', pdbids = names_train, data = data_train)
np.savez_compressed(data_path + '/aggregation/Test_aggregation.npz', pdbids = names_test, data = data_test)
print(data_train.shape)
print(data_test.shape)

(1122, 6, 97)
(277, 6, 97)


# Fuse with the rest of the data

In [8]:
data_path = os.path.dirname(os.getcwd()) + '/data/'

train_data = np.load(data_path + 'aggregation/Train_aggregation.npz')
test_data = np.load(data_path + 'aggregation/Test_aggregation.npz')

tot_train = np.load('/Users/deagogishvili/Documents/PhD/multitask/data/extended/Train_HHblits_extended.npz')
path_tot_test = ['/Users/deagogishvili/Documents/PhD/multitask/data/extended/CASP12_HHblits_extended.npz', 
                 '/Users/deagogishvili/Documents/PhD/multitask/data/extended/TS115_HHblits_extended.npz', 
                 '/Users/deagogishvili/Documents/PhD/multitask/data/extended/CB513_HHblits_extended.npz']

In [9]:
pdbids1 = tot_train['pdbids']
pdbids2 = train_data['pdbids']

pdbids_train = []
pdbids_train.extend(pdbids1)
pdbids_train.extend(pdbids2)

#pdbids_train = tot_train['pdbids']
seq_train = np.concatenate([tot_train['data'], np.zeros((tot_train['data'].shape[0], tot_train['data'].shape[1], 2))], axis = 2) 
train_data_cat = np.concatenate([train_data['data'], np.zeros((train_data['data'].shape[0], 
                                                               seq_train.shape[1] - train_data['data'].shape[1], seq_train.shape[2]))], axis = 1)
extended_train = np.concatenate([seq_train, train_data_cat], axis = 0)

In [31]:
test_sets = []
test_pdbids = []
for i in path_tot_test:
    with np.load(i) as data:
        pdbids_test1 = data['pdbids']
        pdbids_test2 = test_data['pdbids']
        seq_test = np.concatenate([data['data'], np.zeros((data['data'].shape[0], data['data'].shape[1], 2))], axis = 2) 
        test_data_cat = np.concatenate([test_data['data'], np.zeros((test_data['data'].shape[0], 
                                                               seq_test.shape[1] - test_data['data'].shape[1], seq_test.shape[2]))], axis = 1)
        new_set = np.concatenate([seq_test, test_data_cat], axis = 0)
        test_sets.append(new_set)
        
        pdbids_test = []
        pdbids_test.extend(pdbids_test1)
        pdbids_test.extend(pdbids_test2)
        test_pdbids.append(pdbids_test)
        
np.savez_compressed('/Users/deagogishvili/Documents/PhD/multitask/data/extended/Train_extended_aggregation.npz', pdbids = pdbids_train, data = extended_train)
names = ['/Users/deagogishvili/Documents/PhD/multitask/data/extended/CASP12_extended_aggregation.npz', 
         '/Users/deagogishvili/Documents/PhD/multitask/data/extended/TS115_extended_aggregation.npz', 
         '/Users/deagogishvili/Documents/PhD/multitask/data/extended/CB513_extended_aggregation.npz']
for name, data, pdbids in zip(names, test_sets, test_pdbids):
    np.savez_compressed(name, pdbids = pdbids, data = data)